<a href="https://colab.research.google.com/github/PetraNovotna/semisupervised_KES_SVES/blob/master/CinC2020_semisupervised_SVES_KES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prvni textova bunecka.

In [0]:
print("Hello, world!")

Hello, world!


In [0]:
#### mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!wget -O PhysioNetChallenge2020_Training_CPSC.tar.gz https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_CPSC.tar.gz/

--2020-04-22 08:29:25--  https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_CPSC.tar.gz/
Resolving cloudypipeline.com (cloudypipeline.com)... 34.74.104.185
Connecting to cloudypipeline.com (cloudypipeline.com)|34.74.104.185|:9555... connected.
HTTP request sent, awaiting response... 200 
Length: 859926186 (820M) [application/octet-stream]
Saving to: ‘PhysioNetChallenge2020_Training_CPSC.tar.gz’

PhysioNetChallenge2 100%[===================>] 820.09M  15.2MB/s    in 55s     

2020-04-22 08:30:21 (14.8 MB/s) - ‘PhysioNetChallenge2020_Training_CPSC.tar.gz’ saved [859926186/859926186]



In [0]:
!wget -O PhysioNetChallenge2020_Training_CPSC.tar.gz https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_CPSC.tar.gz/ -P /content/drive/My Drive/CinC2020_semisupervised_SVES_KES

--2020-04-22 08:30:55--  https://cloudypipeline.com:9555/api/download/physionet2020training/PhysioNetChallenge2020_Training_CPSC.tar.gz/
Resolving cloudypipeline.com (cloudypipeline.com)... 34.74.104.185
Connecting to cloudypipeline.com (cloudypipeline.com)|34.74.104.185|:9555... connected.
HTTP request sent, awaiting response... 200 
Length: 859926186 (820M) [application/octet-stream]
Saving to: ‘PhysioNetChallenge2020_Training_CPSC.tar.gz’

2020_Training_CPSC. 100%[===================>] 820.09M  15.5MB/s    in 55s     

2020-04-22 08:31:51 (14.9 MB/s) - ‘PhysioNetChallenge2020_Training_CPSC.tar.gz’ saved [859926186/859926186]

--2020-04-22 08:31:51--  http://drive/CinC2020_semisupervised_SVES_KES
Resolving drive (drive)... failed: No address associated with hostname.
wget: unable to resolve host address ‘drive’
FINISHED --2020-04-22 08:31:51--
Total wall clock time: 56s
Downloaded: 1 files, 820M in 55s (14.9 MB/s)


In [0]:
!unzip '/content/drive/My Drive/CinC2020_semisupervised_SVES_KES/PhysioNetChallenge2020_Training_CPSC.tar.gz' -d '/content/drive/My Drive/CinC2020_semisupervised_SVES_KES/data'

Archive:  /content/drive/My Drive/CinC2020_semisupervised_SVES_KES/PhysioNetChallenge2020_Training_CPSC.tar.gz
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/My Drive/CinC2020_semisupervised_SVES_KES/PhysioNetChallenge2020_Training_CPSC.tar.gz or
        /content/drive/My Drive/CinC2020_semisupervised_SVES_KES/PhysioNetChallenge2020_Training_CPSC.tar.gz.zip, and cannot find /content/drive/My Drive/CinC2020_semisupervised_SVES_KES/PhysioNetChallenge2020_Training_CPSC.tar.gz.ZIP, period.


In [7]:
!tar -xvzf '/content/drive/My Drive/CinC2020_semisupervised_SVES_KES/PhysioNetChallenge2020_Training_CPSC.tar.gz' -C '/content/drive/My Drive/CinC2020_semisupervised_SVES_KES/data'

Streaming output truncated to the last 5000 lines.
Training_WFDB/A4378.hea
Training_WFDB/A4379.mat
Training_WFDB/A4379.hea
Training_WFDB/A4380.mat
Training_WFDB/A4380.hea
Training_WFDB/A4381.mat
Training_WFDB/A4381.hea
Training_WFDB/A4382.mat
Training_WFDB/A4382.hea
Training_WFDB/A4383.mat
Training_WFDB/A4383.hea
Training_WFDB/A4384.mat
Training_WFDB/A4384.hea
Training_WFDB/A4385.mat
Training_WFDB/A4385.hea
Training_WFDB/A4386.mat
Training_WFDB/A4386.hea
Training_WFDB/A4387.mat
Training_WFDB/A4387.hea
Training_WFDB/A4388.mat
Training_WFDB/A4388.hea
Training_WFDB/A4389.mat
Training_WFDB/A4389.hea
Training_WFDB/A4390.mat
Training_WFDB/A4390.hea
Training_WFDB/A4391.mat
Training_WFDB/A4391.hea
Training_WFDB/A4392.mat
Training_WFDB/A4392.hea
Training_WFDB/A4393.mat
Training_WFDB/A4393.hea
Training_WFDB/A4394.mat
Training_WFDB/A4394.hea
Training_WFDB/A4395.mat
Training_WFDB/A4395.hea
Training_WFDB/A4396.mat
Training_WFDB/A4396.hea
Training_WFDB/A4397.mat
Training_WFDB/A4397.hea
Training_WFDB